In [1]:
%matplotlib inline
from PIL import Image
from itertools import repeat
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import model_from_json
import keras.callbacks as cb
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def loadModel():
    # load json and create model
    json_file = open('modelD.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("modelD.h5")
    print("Loaded model from disk")
    return loaded_model

In [11]:
loaded_model=loadModel()
loaded_model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

Loaded model from disk


In [4]:
train_dir = "./INRIAPerson/test_64x128_H96"
neg="neg.lst"
pos='pos.lst'
neg=os.path.join(train_dir, neg)
pos=os.path.join(train_dir, pos)
print("neg images are stored at:",neg)
print("pos images are stored at:",pos)

neg images are stored at: ./INRIAPerson/test_64x128_H96/neg.lst
pos images are stored at: ./INRIAPerson/test_64x128_H96/pos.lst


In [5]:
#Create two lists that contain locations of positive and negative images
posList=[]
negList=[]
with open(neg, 'r') as f:
    for line in f:
        line=line[5:].strip('\n')
        line=os.path.join(train_dir, line)
        negList.append(line)
with open(pos, 'r') as f:
    for line in f:
        line=line[5:].strip('\n')
        line=os.path.join(train_dir, line)
        posList.append(line)
#Repeat each item in negList 3 times, for image reuse
# negList = [x for item in negList for x in repeat(item, 3)]

In [6]:
#Load pos images from path specified at posList
def loadPosImage(lst):
    result=[]
    for entry in lst:
        im=Image.open(entry, 'r')
        width, height = im.size   # Get dimensions
        left = (width - new_width)/2
        top = (height - new_height)/2
        right = (width + new_width)/2
        bottom = (height + new_height)/2
        im=im.crop((left, top, right, bottom))
        im=np.asarray(im)
        _,_,z=im.shape
        if(z==4):
            im=np.delete(im,3,axis=2)
        #im=im.reshape(new_width*new_height*3)
        result.append(im)
    return result

def loadNegImage(lst):
    scaleRatio=1.8 #tuned with training dataset
    result=[]
    for entry in lst:
        im=Image.open(entry, 'r')
        baseWidth=im.size[0]
        baseHeight=im.size[1]
        im = im.resize((int(baseWidth/scaleRatio),int(baseHeight/scaleRatio)), Image.ANTIALIAS)
        width, height = im.size   # Get dimensions
        width=width-new_width
        height=height-new_height
        #select top left from available range
        curWidth=0
        curHeight=0
        while(curWidth<width):
            curHeight=0
            while(curHeight<height):
                #print(curWidth, curHeight)
                top=curHeight
                left=curWidth
                img=im.crop((left, top, left+new_width, top+new_height))
                img=np.asarray(img)
                _,_,z=img.shape
                if(z==4):
                    img=np.delete(img,3,axis=2)
                #im=im.reshape(new_width*new_height*3)
                result.append(img)
                #move height 32 pixel a time
                curHeight+=128
            curWidth+=64
    return result

In [7]:
#Hyperparameter
new_width=64
new_height=128
channel=3
posImgList=loadPosImage(posList)
negImgList=loadNegImage(negList)

In [8]:
posLength=len(posImgList)
negLength=len(negImgList)
print(posLength, " pos images, ", negLength, "neg images")
#create tag for each image
posTag=[[0,1]]*posLength
negTag=[[1,0]]*negLength
dataList=[]
dataList.extend(posImgList)
dataList.extend(negImgList)
dataList=np.array(dataList)
#dataList=np.append(posImgList, negImgList, axis = 0)
dataTag=[]
dataTag.extend(posTag)
dataTag.extend(negTag)
dataTag=np.array(dataTag)

1132  pos images,  2496 neg images


In [12]:
score = loaded_model.evaluate(dataList, dataTag, batch_size=len(dataList))

3628/3628 [==============================] - 56s 16ms/step


In [14]:
print(score)

[0.17362354695796967, 0.9498346447944641]
